This notebook explains the functioning of the virtual game mastr chatbot program, written in Python.
Let's start with an overview of the program then we will examine each section in detail.
The purpose of the program is to use ChatGPT as a virtual game master that can take users through text-based adventure games on the web.
This code is for a single player game.

In [ ]:
from flask import Flask, render_template, request, jsonify
import os
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Flask app setup
app = Flask(__name__)

# Environment variable for OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# AI Window Size - Number of moves the AI remembers
AI_WINDOW_SIZE = 20

system_template = """
You are a helpful bot named Bob.

Current conversation:
{history}
Human: {input}
AI Assistant:
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


# Initialize ChatOpenAI with the desired model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize ConversationChain with the appropriate parameters
conversation = ConversationChain(
    llm=chat, 
    prompt=chat_prompt, 
    memory=ConversationBufferWindowMemory(k=AI_WINDOW_SIZE),
    verbose=True
)

@app.route('/')
def home():
    # Render the chat interface
    return render_template('chat.html')

@app.route('/initiate_chat', methods=['GET'])
def initiate_chat():
    # Trigger initial AI response with empty user input
    response = conversation.predict(input="")
    return jsonify({'reply': response})

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json['message']
    response = conversation.predict(input=user_input)
    return jsonify({'reply': response})

if __name__ == '__main__':
    app.run(debug=True)


Lets look at the header, which imports needed libraries.

In [ ]:
from flask import Flask, render_template, request, jsonify
import os
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

The flask library is for running the python program with a web interface.
The os library is for importing system variables.
The langchain library is for interacting with LLMs.
The langchain library is new, so let's talk about it a bit.

langchain.chains.conversation.base.ConversationChain

class langchain.chains.conversation.base.ConversationChain

    Bases: LLMChain

    Chain to have a conversation and load context from memory.

    Example

    from langchain.chains import ConversationChain
    from langchain.llms import OpenAI

    conversation = ConversationChain(llm=OpenAI())

    Create a new model by parsing and validating input data from keyword arguments.

    Raises ValidationError if the input data cannot be parsed to form a valid model.

    param callback_manager: Optional[BaseCallbackManager] = None

        Deprecated, use callbacks instead.

    param callbacks: Callbacks = None

        Optional list of callback handlers (or callback manager). Defaults to None. Callback handlers are called throughout the lifecycle of a call to a chain, starting with on_chain_start, ending with on_chain_end or on_chain_error. Each custom chain can optionally call additional callback methods, see Callback docs for full details.

    param llm: Union[Runnable[LanguageModelInput, str], Runnable[LanguageModelInput, BaseMessage]] [Required]

        Language model to call.

    param llm_kwargs: dict [Optional]

    param memory: langchain.schema.memory.BaseMemory [Optional]

        Default memory store.

    param metadata: Optional[Dict[str, Any]] = None

        Optional metadata associated with the chain. Defaults to None. This metadata will be associated with each call to this chain, and passed as arguments to the handlers defined in callbacks. You can use these to eg identify a specific instance of a chain with its use case.

    param output_parser: BaseLLMOutputParser [Optional]

        Output parser to use. Defaults to one that takes the most likely string but does not change it otherwise.

    param prompt: langchain.schema.prompt_template.BasePromptTemplate = PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

        Default conversation prompt to use.

    param return_final_only: bool = True

        Whether to return only the final parsed result. Defaults to True. If false, will return a bunch of extra information about the generation.

    param tags: Optional[List[str]] = None

        Optional list of tags associated with the chain. Defaults to None. These tags will be associated with each call to this chain, and passed as arguments to the handlers defined in callbacks. You can use these to eg identify a specific instance of a chain with its use case.

    param verbose: bool [Optional]

        Whether or not run in verbose mode. In verbose mode, some intermediate logs will be printed to the console. Defaults to the global verbose value, accessible via langchain.globals.get_verbose().

    __call__(inputs: Union[Dict[str, Any], Any], return_only_outputs: bool = False, callbacks: Optional[Union[List[BaseCallbackHandler], BaseCallbackManager]] = None, *, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, run_name: Optional[str] = None, include_run_info: bool = False) → Dict[str, Any]

        Execute the chain.

        Parameters

                inputs – Dictionary of inputs, or single input if chain expects only one param. Should contain all inputs specified in Chain.input_keys except for inputs that will be set by the chain’s memory.

                return_only_outputs – Whether to return only outputs in the response. If True, only new keys generated by this chain will be returned. If False, both input keys and new keys generated by this chain will be returned. Defaults to False.

                callbacks – Callbacks to use for this chain run. These will be called in addition to callbacks passed to the chain during construction, but only these runtime callbacks will propagate to calls to other objects.

                tags – List of string tags to pass to all callbacks. These will be passed in addition to tags passed to the chain during construction, but only these runtime tags will propagate to calls to other objects.

                metadata – Optional metadata associated with the chain. Defaults to None

                include_run_info – Whether to include run info in the response. Defaults to False.

        Returns

            A dict of named outputs. Should contain all outputs specified in

                Chain.output_keys.



langchain.memory.buffer_window.ConversationBufferWindowMemory

class langchain.memory.buffer_window.ConversationBufferWindowMemory

    Bases: BaseChatMemory

    Buffer for storing conversation memory inside a limited size window.

    Create a new model by parsing and validating input data from keyword arguments.

    Raises ValidationError if the input data cannot be parsed to form a valid model.

    param ai_prefix: str = 'AI'

    param chat_memory: BaseChatMessageHistory [Optional]

    param human_prefix: str = 'Human'

    param input_key: Optional[str] = None

    param k: int = 5

        Number of messages to store in buffer.

    param output_key: Optional[str] = None

    param return_messages: bool = False

    clear() → None

        Clear memory contents.

    classmethod construct(_fields_set: Optional[SetStr] = None, **values: Any) → Model

        Creates a new model setting __dict__ and __fields_set__ from trusted or pre-validated data. Default values are respected, but no other validation is performed. Behaves as if Config.extra = ‘allow’ was set since it adds all passed values

    copy(*, include: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, exclude: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, update: Optional[DictStrAny] = None, deep: bool = False) → Model

        Duplicate a model, optionally choose which fields to include, exclude and change.

        Parameters

                include – fields to include in new model

                exclude – fields to exclude from new model, as with values this takes precedence over include

                update – values to change/add in the new model. Note: the data is not validated before creating the new model: you should trust this data

                deep – set to True to make a deep copy of the model

        Returns

            new model instance

    dict(*, include: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, exclude: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, by_alias: bool = False, skip_defaults: Optional[bool] = None, exclude_unset: bool = False, exclude_defaults: bool = False, exclude_none: bool = False) → DictStrAny

        Generate a dictionary representation of the model, optionally specifying which fields to include or exclude.

    classmethod from_orm(obj: Any) → Model

    classmethod get_lc_namespace() → List[str]

        Get the namespace of the langchain object.

        For example, if the class is langchain.llms.openai.OpenAI, then the namespace is [“langchain”, “llms”, “openai”]

    classmethod is_lc_serializable() → bool

        Is this class serializable?

    json(*, include: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, exclude: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, by_alias: bool = False, skip_defaults: Optional[bool] = None, exclude_unset: bool = False, exclude_defaults: bool = False, exclude_none: bool = False, encoder: Optional[Callable[[Any], Any]] = None, models_as_dict: bool = True, **dumps_kwargs: Any) → unicode

        Generate a JSON representation of the model, include and exclude arguments as per dict().

        encoder is an optional function to supply as default to json.dumps(), other arguments as per json.dumps().

    classmethod lc_id() → List[str]

        A unique identifier for this class for serialization purposes.

        The unique identifier is a list of strings that describes the path to the object.

    load_memory_variables(inputs: Dict[str, Any]) → Dict[str, Any][source]

        Return history buffer.

    classmethod parse_file(path: Union[str, Path], *, content_type: unicode = None, encoding: unicode = 'utf8', proto: Protocol = None, allow_pickle: bool = False) → Model

    classmethod parse_obj(obj: Any) → Model

    classmethod parse_raw(b: Union[str, bytes], *, content_type: unicode = None, encoding: unicode = 'utf8', proto: Protocol = None, allow_pickle: bool = False) → Model

    save_context(inputs: Dict[str, Any], outputs: Dict[str, str]) → None

        Save context from this conversation to buffer.

    classmethod schema(by_alias: bool = True, ref_template: unicode = '#/definitions/{model}') → DictStrAny

    classmethod schema_json(*, by_alias: bool = True, ref_template: unicode = '#/definitions/{model}', **dumps_kwargs: Any) → unicode

    to_json() → Union[SerializedConstructor, SerializedNotImplemented]

    to_json_not_implemented() → SerializedNotImplemented

    classmethod update_forward_refs(**localns: Any) → None

        Try to update ForwardRefs on fields based on this Model, globalns and localns.

    classmethod validate(value: Any) → Model

    property buffer: Union[str, List[langchain.schema.messages.BaseMessage]]

        String buffer of memory.

    property buffer_as_messages: List[langchain.schema.messages.BaseMessage]

        Exposes the buffer as a list of messages in case return_messages is False.

    property buffer_as_str: str

        Exposes the buffer as a string in case return_messages is True.

    property lc_attributes: Dict

        List of attribute names that should be included in the serialized kwargs.

        These attributes must be accepted by the constructor.

    property lc_secrets: Dict[str, str]

        A map of constructor argument names to secret ids.

        For example,

            {“openai_api_key”: “OPENAI_API_KEY”}



langchain.chat_models.openai.ChatOpenAI

class langchain.chat_models.openai.ChatOpenAI

    Bases: BaseChatModel

    OpenAI Chat large language models API.

    To use, you should have the openai python package installed, and the environment variable OPENAI_API_KEY set with your API key.

    Any parameters that are valid to be passed to the openai.create call can be passed in, even if not explicitly saved on this class.

    Example

    from langchain.chat_models import ChatOpenAI
    openai = ChatOpenAI(model_name="gpt-3.5-turbo")

    Create a new model by parsing and validating input data from keyword arguments.

    Raises ValidationError if the input data cannot be parsed to form a valid model.

    param cache: Optional[bool] = None

        Whether to cache the response.

    param callback_manager: Optional[BaseCallbackManager] = None

        Callback manager to add to the run trace.

    param callbacks: Callbacks = None

        Callbacks to add to the run trace.

    param max_retries: int = 6

        Maximum number of retries to make when generating.

    param max_tokens: Optional[int] = None

        Maximum number of tokens to generate.

    param metadata: Optional[Dict[str, Any]] = None

        Metadata to add to the run trace.

    param model_kwargs: Dict[str, Any] [Optional]

        Holds any model parameters valid for create call not explicitly specified.

    param model_name: str = 'gpt-3.5-turbo' (alias 'model')

        Model name to use.

    param n: int = 1

        Number of chat completions to generate for each prompt.

    param openai_api_base: Optional[str] = None

    param openai_api_key: Optional[str] = None

        Base URL path for API requests, leave blank if not using a proxy or service emulator.

    param openai_organization: Optional[str] = None

    param openai_proxy: Optional[str] = None

    param request_timeout: Optional[Union[float, Tuple[float, float]]] = None

        Timeout for requests to OpenAI completion API. Default is 600 seconds.

    param streaming: bool = False

        Whether to stream the results or not.

    param tags: Optional[List[str]] = None

        Tags to add to the run trace.

    param temperature: float = 0.7

        What sampling temperature to use.

    param tiktoken_model_name: Optional[str] = None

        The model name to pass to tiktoken when using this class. Tiktoken is used to count the number of tokens in documents to constrain them to be under a certain limit. By default, when set to None, this will be the same as the embedding model name. However, there are some cases where you may want to use this Embedding class with a model name not supported by tiktoken. This can include when using Azure embeddings or when using one of the many model providers that expose an OpenAI-like API but with different models. In those cases, in order to avoid erroring when tiktoken is called, you can specify a model name to use here.

    param verbose: bool [Optional]

        Whether to print out response text.

    __call__(messages: List[BaseMessage], stop: Optional[List[str]] = None, callbacks: Optional[Union[List[BaseCallbackHandler], BaseCallbackManager]] = None, **kwargs: Any) → BaseMessage

        Call self as a function.

    async abatch(inputs: List[Input], config: Optional[Union[RunnableConfig, List[RunnableConfig]]] = None, *, return_exceptions: bool = False, **kwargs: Optional[Any]) → List[Output]

        Default implementation of abatch, which calls ainvoke N times. Subclasses should override this method if they can batch more efficiently.

    async agenerate(messages: List[List[BaseMessage]], stop: Optional[List[str]] = None, callbacks: Optional[Union[List[BaseCallbackHandler], BaseCallbackManager]] = None, *, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, run_name: Optional[str] = None, **kwargs: Any) → LLMResult

        Top Level call

    async agenerate_prompt(prompts: List[PromptValue], stop: Optional[List[str]] = None, callbacks: Optional[Union[List[BaseCallbackHandler], BaseCallbackManager]] = None, **kwargs: Any) → LLMResult

        Asynchronously pass a sequence of prompts and return model generations.

        This method should make use of batched calls for models that expose a batched API.

        Use this method when you want to:

                take advantage of batched calls,

                need more output from the model than just the top generated value,

                are building chains that are agnostic to the underlying language model

                    type (e.g., pure text completion models vs chat models).

        Parameters

                prompts – List of PromptValues. A PromptValue is an object that can be converted to match the format of any language model (string for pure text generation models and BaseMessages for chat models).

                stop – Stop words to use when generating. Model output is cut off at the first occurrence of any of these substrings.

                callbacks – Callbacks to pass through. Used for executing additional functionality, such as logging or streaming, throughout generation.

                **kwargs – Arbitrary additional keyword arguments. These are usually passed to the model provider API call.

        Returns

            An LLMResult, which contains a list of candidate Generations for each input

                prompt and additional model provider-specific output.

    async ainvoke(input: Union[PromptValue, str, List[BaseMessage]], config: Optional[RunnableConfig] = None, *, stop: Optional[List[str]] = None, **kwargs: Any) → BaseMessage

        Default implementation of ainvoke, which calls invoke in a thread pool. Subclasses should override this method if they can run asynchronously.

    async apredict(text: str, *, stop: Optional[Sequence[str]] = None, **kwargs: Any) → str

        Asynchronously pass a string to the model and return a string prediction.

        Use this method when calling pure text generation models and only the top

            candidate generation is needed.

        Parameters

                text – String input to pass to the model.

                stop – Stop words to use when generating. Model output is cut off at the first occurrence of any of these substrings.

                **kwargs – Arbitrary additional keyword arguments. These are usually passed to the model provider API call.

        Returns

            Top model prediction as a string.

    async apredict_messages(messages: List[BaseMessage], *, stop: Optional[Sequence[str]] = None, **kwargs: Any) → BaseMessage

        Asynchronously pass messages to the model and return a message prediction.

        Use this method when calling chat models and only the top

            candidate generation is needed.

        Parameters

                messages – A sequence of chat messages corresponding to a single model input.

                stop – Stop words to use when generating. Model output is cut off at the first occurrence of any of these substrings.

                **kwargs – Arbitrary additional keyword arguments. These are usually passed to the model provider API call.

        Returns

            Top model prediction as a message.

    async astream(input: Union[PromptValue, str, List[BaseMessage]], config: Optional[RunnableConfig] = None, *, stop: Optional[List[str]] = None, **kwargs: Any) → AsyncIterator[BaseMessageChunk]

        Default implementation of astream, which calls ainvoke. Subclasses should override this method if they support streaming output.

    async astream_log(input: Any, config: Optional[RunnableConfig] = None, *, diff: bool = True, include_names: Optional[Sequence[str]] = None, include_types: Optional[Sequence[str]] = None, include_tags: Optional[Sequence[str]] = None, exclude_names: Optional[Sequence[str]] = None, exclude_types: Optional[Sequence[str]] = None, exclude_tags: Optional[Sequence[str]] = None, **kwargs: Optional[Any]) → Union[AsyncIterator[RunLogPatch], AsyncIterator[RunLog]]

        Stream all output from a runnable, as reported to the callback system. This includes all inner runs of LLMs, Retrievers, Tools, etc.

        Output is streamed as Log objects, which include a list of jsonpatch ops that describe how the state of the run has changed in each step, and the final state of the run.

        The jsonpatch ops can be applied in order to construct state.

    async atransform(input: AsyncIterator[Input], config: Optional[RunnableConfig] = None, **kwargs: Optional[Any]) → AsyncIterator[Output]

        Default implementation of atransform, which buffers input and calls astream. Subclasses should override this method if they can start producing output while input is still being generated.


langchain.prompts.chat.ChatPromptTemplate

class langchain.prompts.chat.ChatPromptTemplate

    Bases: BaseChatPromptTemplate

    A prompt template for chat models.

    Use to create flexible templated prompts for chat models.

    Examples

    from langchain.prompts import ChatPromptTemplate

    template = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ])

    messages = template.format_messages(
        name="Bob",
        user_input="What is your name?"
    )

    Create a new model by parsing and validating input data from keyword arguments.

    Raises ValidationError if the input data cannot be parsed to form a valid model.

    param input_types: Dict[str, Any] [Optional]

        A dictionary of the types of the variables the prompt template expects. If not provided, all variables are assumed to be strings.

    param input_variables: List[str] [Required]

        List of input variables in template messages. Used for validation.

    param messages: List[MessageLike] [Required]

        List of messages consisting of either message prompt templates or messages.

    param output_parser: Optional[BaseOutputParser] = None

        How to parse the output of calling an LLM on this formatted prompt.

    param partial_variables: Mapping[str, Union[str, Callable[[], str]]] [Optional]

    param validate_template: bool = False

        Whether or not to try validating the template.

    async abatch(inputs: List[Input], config: Optional[Union[RunnableConfig, List[RunnableConfig]]] = None, *, return_exceptions: bool = False, **kwargs: Optional[Any]) → List[Output]

        Default implementation of abatch, which calls ainvoke N times. Subclasses should override this method if they can batch more efficiently.

    async ainvoke(input: Input, config: Optional[RunnableConfig] = None, **kwargs: Any) → Output

        Default implementation of ainvoke, which calls invoke in a thread pool. Subclasses should override this method if they can run asynchronously.

    append(message: Union[BaseMessagePromptTemplate, BaseMessage, BaseChatPromptTemplate, Tuple[str, str], Tuple[Type, str], str]) → None[source]

        Append message to the end of the chat template.

        Parameters

            message – representation of a message to append.

    async astream(input: Input, config: Optional[RunnableConfig] = None, **kwargs: Optional[Any]) → AsyncIterator[Output]

        Default implementation of astream, which calls ainvoke. Subclasses should override this method if they support streaming output.

    async astream_log(input: Any, config: Optional[RunnableConfig] = None, *, diff: bool = True, include_names: Optional[Sequence[str]] = None, include_types: Optional[Sequence[str]] = None, include_tags: Optional[Sequence[str]] = None, exclude_names: Optional[Sequence[str]] = None, exclude_types: Optional[Sequence[str]] = None, exclude_tags: Optional[Sequence[str]] = None, **kwargs: Optional[Any]) → Union[AsyncIterator[RunLogPatch], AsyncIterator[RunLog]]

        Stream all output from a runnable, as reported to the callback system. This includes all inner runs of LLMs, Retrievers, Tools, etc.

        Output is streamed as Log objects, which include a list of jsonpatch ops that describe how the state of the run has changed in each step, and the final state of the run.

        The jsonpatch ops can be applied in order to construct state.

    async atransform(input: AsyncIterator[Input], config: Optional[RunnableConfig] = None, **kwargs: Optional[Any]) → AsyncIterator[Output]

        Default implementation of atransform, which buffers input and calls astream. Subclasses should override this method if they can start producing output while input is still being generated.


langchain.prompts.chat.HumanMessagePromptTemplate

class langchain.prompts.chat.HumanMessagePromptTemplate

    Bases: BaseStringMessagePromptTemplate

    Human message prompt template. This is a message sent from the user.

    Create a new model by parsing and validating input data from keyword arguments.

    Raises ValidationError if the input data cannot be parsed to form a valid model.

    param additional_kwargs: dict [Optional]

        Additional keyword arguments to pass to the prompt template.

    param prompt: langchain.prompts.base.StringPromptTemplate [Required]

        String prompt template.

    classmethod construct(_fields_set: Optional[SetStr] = None, **values: Any) → Model

        Creates a new model setting __dict__ and __fields_set__ from trusted or pre-validated data. Default values are respected, but no other validation is performed. Behaves as if Config.extra = ‘allow’ was set since it adds all passed values

    copy(*, include: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, exclude: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, update: Optional[DictStrAny] = None, deep: bool = False) → Model

        Duplicate a model, optionally choose which fields to include, exclude and change.

        Parameters

                include – fields to include in new model

                exclude – fields to exclude from new model, as with values this takes precedence over include

                update – values to change/add in the new model. Note: the data is not validated before creating the new model: you should trust this data

                deep – set to True to make a deep copy of the model

        Returns

            new model instance

    dict(*, include: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, exclude: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, by_alias: bool = False, skip_defaults: Optional[bool] = None, exclude_unset: bool = False, exclude_defaults: bool = False, exclude_none: bool = False) → DictStrAny

        Generate a dictionary representation of the model, optionally specifying which fields to include or exclude.

    format(**kwargs: Any) → BaseMessage[source]

        Format the prompt template.

        Parameters

            **kwargs – Keyword arguments to use for formatting.
        Returns

            Formatted message.

    format_messages(**kwargs: Any) → List[BaseMessage]

        Format messages from kwargs.

        Parameters

            **kwargs – Keyword arguments to use for formatting.
        Returns

            List of BaseMessages.

    classmethod from_orm(obj: Any) → Model

    classmethod from_template(template: str, template_format: str = 'f-string', **kwargs: Any) → MessagePromptTemplateT

        Create a class from a string template.

        Parameters

                template – a template.

                template_format – format of the template.

                **kwargs – keyword arguments to pass to the constructor.

        Returns

            A new instance of this class.

    classmethod from_template_file(template_file: Union[str, Path], input_variables: List[str], **kwargs: Any) → MessagePromptTemplateT

        Create a class from a template file.

        Parameters

                template_file – path to a template file. String or Path.

                input_variables – list of input variables.

                **kwargs – keyword arguments to pass to the constructor.

        Returns

            A new instance of this class.

    classmethod get_lc_namespace() → List[str]

        Get the namespace of the langchain object.

        For example, if the class is langchain.llms.openai.OpenAI, then the namespace is [“langchain”, “llms”, “openai”]

    classmethod is_lc_serializable() → bool

        Return whether or not the class is serializable.

    json(*, include: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, exclude: Optional[Union[AbstractSetIntStr, MappingIntStrAny]] = None, by_alias: bool = False, skip_defaults: Optional[bool] = None, exclude_unset: bool = False, exclude_defaults: bool = False, exclude_none: bool = False, encoder: Optional[Callable[[Any], Any]] = None, models_as_dict: bool = True, **dumps_kwargs: Any) → unicode

        Generate a JSON representation of the model, include and exclude arguments as per dict().

        encoder is an optional function to supply as default to json.dumps(), other arguments as per json.dumps().

    classmethod lc_id() → List[str]

        A unique identifier for this class for serialization purposes.

        The unique identifier is a list of strings that describes the path to the object.

    classmethod parse_file(path: Union[str, Path], *, content_type: unicode = None, encoding: unicode = 'utf8', proto: Protocol = None, allow_pickle: bool = False) → Model

    classmethod parse_obj(obj: Any) → Model

    classmethod parse_raw(b: Union[str, bytes], *, content_type: unicode = None, encoding: unicode = 'utf8', proto: Protocol = None, allow_pickle: bool = False) → Model

    classmethod schema(by_alias: bool = True, ref_template: unicode = '#/definitions/{model}') → DictStrAny

    classmethod schema_json(*, by_alias: bool = True, ref_template: unicode = '#/definitions/{model}', **dumps_kwargs: Any) → unicode

    to_json() → Union[SerializedConstructor, SerializedNotImplemented]

    to_json_not_implemented() → SerializedNotImplemented

    classmethod update_forward_refs(**localns: Any) → None

        Try to update ForwardRefs on fields based on this Model, globalns and localns.

    classmethod validate(value: Any) → Model

    property input_variables: List[str]

        Input variables for this prompt template.

        Returns

            List of input variable names.

    property lc_attributes: Dict

        List of attribute names that should be included in the serialized kwargs.

        These attributes must be accepted by the constructor.

    property lc_secrets: Dict[str, str]

        A map of constructor argument names to secret ids.

        For example,

            {“openai_api_key”: “OPENAI_API_KEY”}



Now let's look at the next part of the code:

In [ ]:
# Flask app setup
app = Flask(__name__)

# Environment variable for OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# AI Window Size - Number of moves the AI remembers
AI_WINDOW_SIZE = 20

These lines declare a flask app, define the api_key from the environment variable, and sets the chat window size for holding the chat history in a rolling memory window containing the last 20 chats.

The next section defines a LLM prompt template.  The template contains instructions for the bot followed by the chat history and the user's input for this turn.

In [ ]:
system_template = """
You are a helpful bot named Bob.

Current conversation:
{history}
Human: {input}
AI Assistant:
"""

The next section builds the prompts from the templates.

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template="{input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

The next section initializes the chat bot and defines a conversation chain.

# Initialize ChatOpenAI with the desired model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Initialize ConversationChain with the appropriate parameters
conversation = ConversationChain(
    llm=chat, 
    prompt=chat_prompt, 
    memory=ConversationBufferWindowMemory(k=AI_WINDOW_SIZE),
    verbose=True
)

The final section contains the routes 

In [ ]:
@app.route('/')
def home():
    # Render the chat interface
    return render_template('chat.html')

@app.route('/initiate_chat', methods=['GET'])
def initiate_chat():
    # Trigger initial AI response with empty user input
    response = conversation.predict(input="")
    return jsonify({'reply': response})

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json['message']
    response = conversation.predict(input=user_input)
    return jsonify({'reply': response})

if __name__ == '__main__':
    app.run(debug=True)

By changing the system template, I can instruct the bot to behave in defferent ways, depending on the game state.